MERGE FILTERED UMLS PV CUI BACK WITH FITBIR DATA ELEMENT REPORT


<br>
This script will incorporate the new UMLS Permissible Value (PV) concept attributes into a data element report so that it can be imported into BRICS.<br>
DESCRIPTION:<br>
    <br>
(1): Takes PV_step1_XYZ_PVDs_exploded_UMLS_keepCol.xlsx file and filters rows that are manually marked in the KEEP column<br>
(2): Merges manually curated UMLS PV CUI set back with original dataElementExport.csv file<br>
(3): Creates summary file so user can check if any PV cuis are missing<br>
NEW COLUMNS IN DATA ELEMENT IMPORT: <br>
    <br>
1) *data element concept names: 3 concept names max per CDE, e.g. "Traumatic Brain Injury | neuropathology disease| Neuropathology<br>
2) *data element concept identifiers: 3 concepts (name+CUI) max per CDE, e.g., "Traumatic Brain Injury (C0876926)| neuropathology disease (C1540677)| Neuropathology (C0876934)<br>
3) *data element terminology sources: e.g. UMLS, NCIt, SNOMED, HL7, LOINC, RxNORM, etc.<br>
4) permissible value concept names: e.g. "Traumatic Brain Injury | neuropathology disease| Neuropathology <br>
5) permissible value concept identifiers: each PV must be assigned only one pair "concept names and CUI", e.g., No(C1298908)|Not Assessed(C3846720)|Unknown(C0439673)|Yes (indicator)(C1705108)<br>
6) permissible value terminology sources: the source for data element concept(s), e.g. UMLS, NCIt, SNOMED, HL7, LOINC, RxNORM, etc.<br>
    <br>
* these columns will only be included in the ouput if the DE scripts have already been run and the generated columns are <br>
part of the DE_report tab in PV_step1_XYZ_PVDs_exploded_UMLS_keepCol.xlsx<br>


LOAD PACKAGES

In [14]:
import cui_mapping_functions as cmf # contains custom functions for loading/processing FITBIR csv datafiles
from check_missing_cuis import count_sep, count_PV_CUIs_missing, count_CUIs_NA, count_CUIs_per_sep, returnFlaggedCUIs
from functools import partial
import pandas as pd
import numpy as np
import os
import tkinter as tk  # for dialog boxes
from tkinter import filedialog
import importlib
root = tk.Tk()
root.withdraw()
importlib.reload(cmf)

<module 'cui_mapping_functions' from 'C:\\Users\\armengolkm\\Documents\\BRICS\\Sourcetree_BRICS\\Operations\\UMLS Coding\\UMLS-CUI_DE-PV_pipeline_ipynb_Version-1.5_2022-12-06\\cui_mapping_functions.py'>

INITIALIZATION

In [15]:
verify_cuis = 1  # 0 to skip running check_missing_cuis.py functions
sourceTerm = 'UMLS'  # set value of source terminology for data element import

LOAD "PV_step1_XYZ_PVDs_exploded_UMLS_keepCol.xlsx" file

In [16]:
fp_xlsx_file = filedialog.askopenfilename(
    title='Choose xlsx file "PV_step1_XYZ_PVDs_exploded_UMLS_keepCol"')  # file with PV rows marked for keeping
sheet1 = 'PVDs_UMLS'
sheet2 = 'DE_report'
sheet3 = 'DE_report_PVDs_exploded'
df_PVDs_UMLS = pd.read_excel(
    fp_xlsx_file, sheet_name=sheet1, header=0, keep_default_na=False)
df_DE_report = pd.read_excel(
    fp_xlsx_file, sheet_name=sheet2, header=0, keep_default_na=False)
df_DE_report_PVDs_exploded = pd.read_excel(
    fp_xlsx_file, sheet_name=sheet3, header=0, keep_default_na=False)

CREATE STEP 2 DIRECTORY

In [17]:
dir_parent = os.path.dirname(os.path.dirname(fp_xlsx_file))
# creates new files with unneeded/irrelevant PV CUIs removed
dir_step2 = cmf.create_folder(os.path.join(
    dir_parent, 'PV_Step-2_Filter UMLS CUIs and Merge with DE Report'))

REMOVE UNNEEDED PV UMLS CODES

In [18]:
df_PVDs_UMLS['keep'].replace(r'^\s*$', np.nan, regex=True, inplace=True) # makes sure any empty spaces are nan and won't be removed in next step
df_PVDs_UMLS_filtered = df_PVDs_UMLS[df_PVDs_UMLS['keep'].notnull()].reset_index(
    drop=True)

CONCATENATE PVDs WITH MULTIPLE CUIS

In [19]:
df_PVDs_mult = df_PVDs_UMLS_filtered.groupby(["variable name",
                                              "PVDs_exploded"])[['permissible value concept identifiers',
                                                                 'permissible value concept names']].agg('/'.join)  # aggregate multiple PVD CUIs and separate by '/'
df_sourceTerm_mult = df_PVDs_UMLS_filtered.groupby(["variable name", "PVDs_exploded"])[
    ['permissible value terminology sources']].agg('/'.join)  # aggregate multiple source terminologies and separate by '/'
df_temp0 = pd.merge(df_PVDs_mult, df_sourceTerm_mult, on=[
                    "variable name", "PVDs_exploded"])

MERGE UNIQUE PV CODES WITH FITBIR DATA ELEMENT REPORT

In [20]:
df_new = pd.merge(df_temp0, df_DE_report_PVDs_exploded, how="right",
                  left_on=["variable name", "PVDs_exploded"],
                  right_on=["variable name", "PVDs_exploded"])  # merge PV/UMLS pairings with exploded PVDs file that contains all PVDs (excludes DEs w/o PVDs)
df_new.drop_duplicates(inplace=True)
df_new = df_new.fillna('')
df_new = df_new[['variable name', 'permissible values', 'permissible value descriptions', 'permissible value terminology sources',
                 'permissible value concept names', 'permissible value concept identifiers']].copy()

In [21]:
source = df_new.groupby(['variable name', 'permissible values'])['permissible value terminology sources'].agg(
    '|'.join).reset_index(name='permissible value terminology sources')
# source['permissible value terminology sources'] = sourceTerm # this line changes all terminology sources into source specified at beginning. comment out if you'd like '|' separated sources for each PVD
source['permissible value terminology sources'].replace(
    {'MTH': 'UMLS'}, inplace=True)
#pvd=df_new.groupby(['variable name', 'permissible values'])['permissible value descriptions'].agg('|'.join).reset_index(name='permissible value descriptions')
cui = df_new.groupby(['variable name', 'permissible values'])['permissible value concept identifiers'].agg(
    '|'.join).reset_index(name='permissible value concept identifiers')
# url=df_new.groupby(['variable name', 'permissible values'])['url'].agg('|'.join).reset_index(name='url')
result = df_new.groupby(['variable name', 'permissible values'])['permissible value concept names'].agg(
    '|'.join).reset_index(name='permissible value concept names')

In [22]:
df_temp1 = pd.merge(source, cui, on=["variable name", "permissible values"])
# df_temp1 = pd.merge(pvd, cui, on=["variable name","permissible values"])
# df_temp2 = pd.merge(df_temp1, url, on=["variable name","permissible values"])
df_final_temp = pd.merge(df_temp1, result, on=[
                         "variable name", "permissible values"])
df_final_temp['permissible value terminology sources'] = df_final_temp['permissible value terminology sources'].apply(lambda x: '|'.join([sourceTerm for i in x.split('|')]))

df_final = pd.merge(df_DE_report, df_final_temp, how="left",
                    on="variable name", suffixes=('', '_y'))

In [23]:
col_reorder = ["variable name", "title",
               "data element concept names",  "data element concept identifiers", "data element terminology sources",
               "element type", "definition", "short description", "datatype", "maximum character quantity", "input restriction",
               "minimum value", "maximum value",
               "permissible values", "permissible value descriptions", "permissible value output codes",
               "permissible value concept names", "permissible value concept identifiers", "permissible value terminology sources",
               "unit of measure", "guidelines/instructions", "notes", "preferred question text", "keywords", "references", "historical notes", "see also",
               "effective date", "until date",
               "population.all", "domain.general (for all diseases)", "domain.traumatic brain injury", "domain.Parkinson's disease", "domain.Friedreich's ataxia", "domain.stroke",
               "domain.amyotrophic lateral sclerosis", "domain.Huntington's disease", "domain.multiple sclerosis", "domain.neuromuscular diseases", "domain.myasthenia gravis",
               "domain.spinal muscular atrophy", "domain.Duchenne muscular dystrophy/Becker muscular dystrophy", "domain.congenital muscular dystrophy", "domain.spinal cord injury",
               "domain.headache", "domain.epilepsy",
               "classification.general (for all diseases)", "classification.acute hospitalized", "classification.concussion/mild TBI", "classification.epidemiology", "classification.moderate/severe TBI: rehabilitation",
               "classification.Parkinson's disease", "classification.Friedreich's ataxia", "classification.stroke", "classification.amyotrophic lateral sclerosis", "classification.Huntington's disease",
               "classification.multiple sclerosis", "classification.neuromuscular diseases", "classification.myasthenia gravis", "classification.spinal muscular atrophy",
               "classification.Duchenne muscular dystrophy/Becker muscular dystrophy", "classification.congenital muscular dystrophy", "classification.spinal cord injury", "classification.headache",
               "classification.epilepsy",  "Label(s)", "submitting organization name", "submitting contact name", "submitting contact information",
               "steward organization name", "steward contact name", "steward contact information"]

In [24]:
cols_de = ["data element concept names",
           "data element concept identifiers", "data element terminology sources"]
# will reorder columns to align with DE import template. if/else statement if new DE columns aren't present yet
if all(col in df_final.columns for col in cols_de):
    df_final = df_final[col_reorder]
else:
    col_reorder = [col for col in col_reorder if col not in cols_de]
    df_final = df_final[col_reorder]

SAVE FINALIZED IMPORT TEMPLATE

In [25]:
filename = os.path.basename(fp_xlsx_file).split('_')[1]
fp_step2 = f"{dir_step2}/PV_step2_{filename}_dataElementImport_cuis.csv"
df_final.to_csv(fp_step2, index=False)  # output df_final dataframe to csv

VERIFY NUMBER OF CUIS STEPS

In [26]:
if verify_cuis:

    ## select relevant columns
    df_verify = df_final[['variable name', 'title', 'definition', 'permissible values',
                          'permissible value descriptions', 'permissible value output codes', 'permissible value concept names',
                          'permissible value concept identifiers', 'permissible value terminology sources']].copy()

    ## pv verification
    n_pvds, pvds = zip(*[(x[0], x[1]) for x in list(map(partial(count_sep, sep="|"),
                       df_verify['permissible value descriptions']))])
    n_missing, idx_missing = zip(*[(x[0], x[1]) for x in list(
        map(count_PV_CUIs_missing, df_verify['permissible value concept identifiers']))])
    n_NA, idx_NA = zip(*[(x[0], x[1]) for x in list(map(count_CUIs_NA,
                       df_verify['permissible value concept identifiers']))])
    n_mult, idx_mult_cui = zip(*[(x[0], x[1]) for x in list(
        map(partial(count_CUIs_per_sep, sep1="|", sep2="/"), df_verify['permissible value concept identifiers']))])

    ls_pvd_missing_cui = returnFlaggedCUIs(pvds, idx_missing)
    ls_pvd_cui_NA = returnFlaggedCUIs(pvds, idx_NA)
    ls_pvd_mult_cui = returnFlaggedCUIs(pvds, idx_mult_cui)

    n_pvd_missing_cui = list(map(lambda x: len(x) if type(
        x) is list else np.nan, ls_pvd_missing_cui))
    n_pvd_cui_NA = list(map(lambda x: len(x) if type(x)
                        is list else np.nan, ls_pvd_cui_NA))
    n_pvd_mult_cui = list(map(lambda x: len(x) if type(
        x) is list else np.nan, ls_pvd_mult_cui))

    ## add verification output as columns
    df_verify['pvds missing cui'] = ls_pvd_missing_cui
    df_verify['total # pvds missing cui'] = n_pvd_missing_cui
    df_verify['pvds cuis Not Available'] = ls_pvd_cui_NA
    df_verify['total # cuis Not Available'] = n_pvd_cui_NA
    df_verify['pvds with multiple cuis'] = ls_pvd_mult_cui
    df_verify['total # of pvds with multiple cuis'] = n_pvd_mult_cui

    ## write verification file
    fp_step2_verify = f"{dir_step2}/PV_step2_{filename}_check-missing_cui.csv"
    df_verify.to_csv(fp_step2_verify, index=False)